# PROJET : Implémentation un modèle de scoring (Data Exploration)

Pour ce projet, nous nous plaçons en tant que **Data Scientist** au sein d'une société financière proposant des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt. 

Notre entreprise souhaite développer un **modèle de scoring de la probabilité de défaut de paiement du client**. Aussi, nous voulons créer un **dashboard interactif** pour que les chargés de relation client puissent expliquer de façon la plus transparente possible les décisions d’octroi de crédit.


Ce projet sera constitué de 4 parties distinctes :
- Data Exploration (**Stack technique:** `Plotly`)
- Data Cleaning & Preprocessing (**Stack technique:** `Sklearn`)
- Modelization (**Stack technique:** `Regression Logistique`, `Random Forest`, `LightGBM`, `SMOTE`, `Custom score`)
- Dashboarding (**Stack technique:** `Streamlit`, `Plotly`, `Docker`, `Google Cloud Platform`)


L'ensemble des fonctions utilisées pour ce projet sont définis dans des fichiers utils.py

## Partie 1 : Data Exploration

Ce Notebook réalise un preprocessing et une exploration des données.
On accorde une importance particulière à la phase de cleaning des données du fait du **déséquilibre des classes** sur les targets.

### Import Packages

In [ ]:
# Classic lib
import numpy as np
import pandas as pd 

# Data viz Lib.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# File system management.
import os

# Import custom functions.
from Utils.utils_preprocessing import *

## Settings

In [ ]:
# Format & option.
sns.set(rc={'figure.figsize':(16,9)})
pd.options.display.max_columns = 150

# Style use.
sns.set_style('darkgrid')
sns.set_context("notebook", 
                font_scale=1.5, 
                rc={"lines.linewidth": 2.5})
plt.style.use('ggplot')

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# Import Data 

Les données sont composées de **9 fichiers**, un dataset principal d'entrainement (avec les targets), un de test, 7 autres apportant des informations supplémentaires sur les clients.


### Dataset dimension :  

307511 observations, 122 features  

### Features : 

- Chaque prêt est spécifié par son **SK_ID_CURR**
- Chaque prêt à sa **TARGET** (0 : `pas de défaillance` et 1 : `défaillance de paiement`)
- **FLAG_OWN_CAR** : possède sa voiture ou non 
- **AMT_INCOME_TOTAL** : salaire annuel total
- **AMT_CREDIT** : montant total du crédit
- **DAYS_EMPLOYED** : nbr de jours travaillés
- **EXT_SOURCE_1/2/3** : sources extérieures (score entre 0 et 1)

In [ ]:
# Dataset import.
df_app_train = pd.read_csv('Data/application_train.csv')
df_app_train.head()

# Check Columns Types

On regarde le type des features, `int`, `float` ou `object` (variables catégoriques)

In [ ]:
# Check data types.
df_app_train.dtypes.value_counts()

In [ ]:
# Check the number of unique value for categorical features.
df_app_train.select_dtypes('object').apply(pd.Series.nunique, axis=0)

In [ ]:
# Check NaN on categorical features
df_app_train.select_dtypes('object').isna().sum()

In [ ]:
# Table on missing values
missing_values = missing_values_table(df_app_train)
missing_values.head(25)

In [ ]:
show_miss_values(df_app_train)

# Data Exploration

Pour la partie exploration on se focalisera sur les aspects suivant :
- Analyse sur le déséquilibre des classes
- Repérer des variables scindant bien les populations d'individus postifs et négatifs

In [ ]:
# Figure 1.
fig_tot = make_subplots(rows=1, cols=2,
                        specs=[[{"type": "domain"}, {"type": "bar"}]],
                        subplot_titles=["Type of loans", "Applicants jobs"])


fig_tot.add_trace(go.Pie(labels=df_app_train["NAME_CONTRACT_TYPE"].value_counts().index, 
                         values=df_app_train["NAME_CONTRACT_TYPE"].value_counts(), hole=.5, legendgroup = '1'),row=1, col=1)
fig_tot.add_trace(go.Bar(x=df_app_train.NAME_EDUCATION_TYPE.dropna().value_counts().index, 
                         y=df_app_train.NAME_EDUCATION_TYPE.dropna().value_counts(), showlegend=False, marker= dict(color="rgb(65, 105, 225)")), row=1, col=2, )

#fig_tot.update_xaxes(tickangle=-90)
fig_tot.update_layout(title="Applicants informations", width=950, height=400, 
                      template="plotly_dark", legend_tracegroupgap=15, xaxis=dict(tickangle=20))
fig_tot.show()

# Figure 2.
fig = go.Figure()
fig.add_trace(go.Bar(x=df_app_train.OCCUPATION_TYPE.dropna().value_counts().index,
                     y=df_app_train.OCCUPATION_TYPE.dropna().value_counts()))
fig.update_layout(title="Occupation Type", width=950, height=350, template="plotly_dark")

In [ ]:
# Specify settings of color for exploration.
pos_color = 'rgb(225, 127, 0)'
neg_color = 'rgb(65, 105, 225)'

# Split the initial DF in positive and negative target.
df_pos, df_neg  = df_app_train[df_app_train.TARGET == 1], df_app_train[df_app_train.TARGET == 0]

In [ ]:
df_pos.AMT_CREDIT = df_pos.AMT_CREDIT.apply(lambda x : np.round(x / 100000, 0) * 100000)
df_pos_aggregated = pd.DataFrame(df_pos.groupby("AMT_CREDIT").count().TARGET)

df_neg.AMT_CREDIT = df_neg.AMT_CREDIT.apply(lambda x : np.round(x / 100000, 0) * 100000)
df_neg_aggregated = pd.DataFrame(df_neg.groupby("AMT_CREDIT").count().TARGET)

df_pos_aggregated.TARGET = df_pos_aggregated.TARGET.apply(lambda x : x*100/df_pos_aggregated.TARGET.sum())
df_neg_aggregated.TARGET = df_neg_aggregated.TARGET.apply(lambda x : x*100/df_neg_aggregated.TARGET.sum())

In [ ]:
df_count_label = pd.DataFrame(df_app_train.TARGET.value_counts())


# Figure 1.
fig_tot = make_subplots(rows=1, 
                        cols=2,
                        specs=[[{"type": "bar"}, {"type": "domain"}]],
                        )

fig_tot.add_trace(go.Bar(x = ["0"], y = [df_count_label.TARGET[0]], name="0", marker_color=neg_color), row=1, col=1)
fig_tot.add_trace(go.Bar(x = ["1"], y = [df_count_label.TARGET[1]], name="1", marker_color=pos_color), row=1, col=1)
fig_tot.add_trace(go.Pie(labels=df_count_label.index.get_level_values(0), 
                         values=df_count_label.TARGET, marker=dict(colors=[neg_color, pos_color])),row=1, col=2)

fig_tot.update_layout(title="Check of imbalanceness", width=950, height=400, template="plotly_dark", showlegend=False)

fig_tot.show()



# Figure 2.
fig = go.Figure()
fig.add_trace(go.Bar(x=df_pos_aggregated.index,
                     y=list(df_pos_aggregated.TARGET),
                     name='1',
                     marker_color=pos_color
                ))

fig.add_trace(go.Bar(x=df_neg_aggregated.index,
                     y=list(df_neg_aggregated.TARGET),
                     name='0',
                     marker_color=neg_color
                ))

fig.update_layout(xaxis_tickfont_size=14,
                  xaxis_range=[0, 1500000],
                  yaxis=dict(title='%',
                             titlefont_size=16,
                             tickfont_size=14),
                  xaxis=dict(title='Credit amount'),
                  legend=dict(x=0,
                              y=1.0,
                              bgcolor='rgba(255, 255, 255, 0)',
                              bordercolor='rgba(255, 255, 255, 0)'),
                  barmode='group',
                  bargap=0.15,
                  bargroupgap=0.1)

fig.update_layout(width=950, height=350, template="plotly_dark")
fig.show()

In [ ]:
# Binarize every 5 years.
age_data = df_app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / -365
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data_pos_groups, age_data_neg_groups  = age_data[age_data.TARGET == 1].groupby('YEARS_BINNED').count(), age_data[age_data.TARGET == 0].groupby('YEARS_BINNED').count()

age_data_pos_groups["interval"], age_data_neg_groups["interval"] = age_data_pos_groups.index, age_data_neg_groups.index,
age_data_pos_groups["interval"] = age_data_pos_groups["interval"].apply(lambda x: str([int(x.left), int(x.right)]))
age_data_neg_groups["interval"] = age_data_neg_groups["interval"].apply(lambda x: str([int(x.left), int(x.right)]))

age_data_pos_groups["TARGET"] = age_data_pos_groups["TARGET"].apply(lambda x: (x*100)/df_pos.shape[0])
age_data_neg_groups["TARGET"] = age_data_neg_groups["TARGET"].apply(lambda x: (x*100)/df_neg.shape[0])

In [ ]:
fig = make_subplots(rows=1, 
                    cols=2,
                    specs=[[{"type": "xy"}, {"type": "xy"}]],
                    subplot_titles=['Age distribution', 'Age on positive and negative target'])

fig.add_trace(go.Histogram(x=-df_app_train.DAYS_BIRTH/365, nbinsx=75, showlegend=False), row=1, col=1)

fig.add_trace(go.Box(x=-df_neg.DAYS_BIRTH / 365, name='0', marker_color=neg_color), row=1, col=2)
fig.add_trace(go.Box(x=-df_pos.DAYS_BIRTH / 365, name='1', marker_color=pos_color), row=1, col=2)

fig.update_layout(width=950, height=350, template="plotly_dark")
fig.show()

# Show impact of age on target.
fig_tot = go.Figure()

fig_tot.add_trace(go.Bar(x=list(age_data_pos_groups.interval), 
                     y=list(age_data_pos_groups.TARGET),
                     name='1',
                     marker_color=pos_color))

fig_tot.add_trace(go.Bar(x=list(age_data_neg_groups.interval), 
                     y=list(age_data_neg_groups.TARGET),
                     name='0',
                     marker_color=neg_color
                ))

fig_tot.update_layout(xaxis_tickfont_size=14,
                  yaxis=dict(title='%',
                             titlefont_size=16,
                             tickfont_size=14),
                  xaxis=dict(title='Groupe of ages'),
                  legend=dict(x=0,
                              y=1,
                              bgcolor='rgba(255, 255, 255, 0)',
                              bordercolor='rgba(255, 255, 255, 0)'),
                  barmode='group',
                  bargap=0.15,
                  bargroupgap=0.1)

fig_tot.update_layout(width=950, height=350, template="plotly_dark")
fig_tot.show()

In [ ]:
# Remove an outlier.
df_pos = df_pos[df_pos.DAYS_EMPLOYED != 365243]
df_neg = df_neg[df_neg.DAYS_EMPLOYED != 365243]

fig_1 = go.Figure()
 
fig_1.add_trace(go.Box(x=-df_neg.DAYS_EMPLOYED, name='0', marker_color=neg_color))
fig_1.add_trace(go.Box(x=-df_pos.DAYS_EMPLOYED, name='1', marker_color=pos_color))
fig_1.update_layout(title="Days employed", width=900, height=300, template="plotly_dark")
fig_1.show()


fig_tot = make_subplots(rows=1, 
                        cols=3,
                        specs=[[{"type": "xy"}, {"type": "xy"}, {"type": "xy"}]],
                        subplot_titles=['External source 1', 'External source 2', 'External source 3'])


# Boxplot 1.
fig_tot.add_trace(go.Box(y=-df_neg.EXT_SOURCE_1, name='0', marker_color=neg_color), row=1, col=1)
fig_tot.add_trace(go.Box(y=-df_pos.EXT_SOURCE_1, name='1', marker_color=pos_color), row=1, col=1)

# Boxplot 2.
fig_tot.add_trace(go.Box(y=-df_neg.EXT_SOURCE_2, name='0', marker_color=neg_color), row=1, col=2)
fig_tot.add_trace(go.Box(y=-df_pos.EXT_SOURCE_2, name='1', marker_color=pos_color), row=1, col=2)

# Boxplot 3.
fig_tot.add_trace(go.Box(y=-df_neg.EXT_SOURCE_3, name='0', marker_color=neg_color), row=1, col=3)
fig_tot.add_trace(go.Box(y=-df_pos.EXT_SOURCE_3, name='1', marker_color=pos_color), row=1, col=3)

fig_tot.update_layout(title="External sources", width=900, height=400, template="plotly_dark", showlegend=False)
fig_tot.show()


## Analyse de corrélation bivariée avec les variables continues

- ANOVA Test : 

H0 : Tous les échantillons ont la même moyenne.  
H1 : L'hypothèse alternative est qu'au moins l'un d'eux joue les trouble-fête avec une moyenne sensiblement différente des autres.

Le résultat d'ANOVA est la « statistique F ». Ce ratio montre la différence entre la variance à l'intérieur du groupe et la variance entre les groupes, ce qui produit finalement un chiffre qui permet de conclure que l'hypothèse nulle est soutenue ou rejetée. S'il existe une différence significative entre les groupes, l'hypothèse nulle n'est pas soutenue, et le ratio F sera plus grand.

La p-valeur est une mesure de référence pour décider du rejet ou non de H0  
-> Si p < 0.05 on peut rejetter H0 avec un certain degré de confiance

In [ ]:
df_continuous_cor = check_correlation_classif_task_continuous(df_app_train, 'TARGET')
df_continuous_cor.head(10)

In [ ]:
df_continuous_cor.tail(10)

## Analyse de corrélation avec les variables catégoriques

- Test du chi-deux -> H0 : les deux variables testées sont indépendantes
- Calcul du V de Cramer (Hypothèse : variables catégoriques avec + de 2 modalités)  

Plus V est proche de zéro, moins les variables étudiées sont dépendantes. Au contraire, donc, il vaudra 1 lorsque les deux variables sont complètement dépendantes.

Conclusion : Il y a une dépendance mais peu significative

In [ ]:
df_cat_cor = check_correlation_classif_task_categorical(df_app_train, "TARGET")
df_cat_cor